# Scraping - Portal inmobiliario

Con el objetivo de realizar análisis y construir modelos de Machine Learning, se realiza scraping de la página https://www.icasas.cl
. iCasas permite acceso razonable según su robots.txt. Dado que la localización exacta de las propiedades no está disponible por privacidad, se extrae y utiliza la comuna y la provincia de cada anuncio como variables.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import random

# CONFIGURACIÓN
url_base = "https://www.icasas.cl"
Objetivo_url = "https://www.icasas.cl/venta/casas/valparaiso-region-v/list"

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}


def obtener_links_listado(Paginas, Muestra):    
    
    # Creación lista para guardar información.
    Propiedades= []

    Lista_paginas = list(range(1,Paginas + 1))
    Muestra_aleatoria = random.sample(Lista_paginas, Muestra)
    # Navegación hasta la pagina deseada para scrapping.
    
    for pagina in sorted(Muestra_aleatoria):
        if pagina == 1:
            url= Objetivo_url
        else:
            url = f"{Objetivo_url}/p_{pagina}"
        response = requests.get(url)    
        # Manejo de errores

        if response.status_code != 200:
            print(f"Error accediendo a pagina: {pagina}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
    
        for link in soup.find_all('a', class_='detail-redirection'):
            href = link.get('href')
            if href:
                propiedad_url = urljoin(url_base, href)
                if propiedad_url not in Propiedades:
                    Propiedades.append(propiedad_url)
    print(f'Se obtuvo {len(Propiedades)} propiedades')

    return Propiedades

In [3]:
links = obtener_links_listado(100, 67)

links

Se obtuvo 996 propiedades


['https://www.icasas.cl/propiedad/b4c4-ae6b-194a9bc-400a0f616db0-7723',
 'https://www.icasas.cl/propiedad/6344-a796-7cabb976-36a02a0d33ad-3825',
 'https://www.icasas.cl/propiedad/c5c8-905f-198f811-e8ce324c3c89-78e6',
 'https://www.icasas.cl/propiedad/da21-9ccf-197c21a-9893f8b45585-73cd',
 'https://www.icasas.cl/propiedad/d725-a794-19b8b7b-9e90fbbb8acf-7553',
 'https://www.icasas.cl/propiedad/2429-b0ac-19b8766-b51afd16cf7c-77ab',
 'https://www.icasas.cl/propiedad/b1df-98d0-f1e28d5-c4e0a96b4996-4a4c',
 'https://www.icasas.cl/propiedad/5e59-85e0-704f482-fa9d9f282638-4507',
 'https://www.icasas.cl/propiedad/c669-88ff-95bacc67-9df7974a8302-46fd',
 'https://www.icasas.cl/propiedad/a2cf-af13-199bfc6-7e383bc7d7b7-72b1',
 'https://www.icasas.cl/propiedad/69ac-b70c-19a1335-2e1a5f523e95-7494',
 'https://www.icasas.cl/propiedad/c71f-abea-196b8a8-4462e4aaf1fe-8080',
 'https://www.icasas.cl/propiedad/67f6-ae36-19baa31-cbec9534a6a5-74d2',
 'https://www.icasas.cl/propiedad/4540-916a-197c6e2-f8dac1f5d8

In [4]:
def scrapping_por_link(url):

    print(f'Obteniendo información de: {url}')
    response = requests.get(url, headers=HEADERS)

    # Manejo de errores
    if response.status_code != 200:
        print("Error accediendo al portal")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Diccionario para guardar información
    Datos={}

    # Precio
    Precio = soup.find_all('div', class_='price')
    if Precio:
        Datos['Precio UF'] = Precio[0].text.split()[1]

    # Numero habitaciones
    habitaciones = soup.find_all('li', class_='bedrooms')
    if habitaciones:
        Datos['N Habitaciones'] = habitaciones[0].text.split()[0]

    # Metros cuadrados (Generalmente terreno, varia segun publicacion. Comparar con descripción)
    Area = soup.find_all('li', class_='dimensions')
    if Area:
        Datos['Area m2']=Area[0].text.split()[0].replace("m2", "")

    # Cantidad de baños
    Baños = soup.find_all('li', class_='bathrooms')
    if Baños:
        Datos['N Baños']=Baños[0].text.split()[0]

    # Comuna
    Localizacion = soup.find('input', id='locationShown') 
    if Localizacion:
        Ubicaciones = Localizacion.get('value').split(', ')
        Comuna = Ubicaciones[0]
        Provincia = Ubicaciones[1]
        Datos['Comuna'] = Comuna
        Datos['Provincia'] = Provincia

    # Caracteristicas interiores
    Caracteristicas = soup.find_all('li', class_='tick')
    for n in Caracteristicas:
        texto = n.text.strip()
        
        if ':' in texto:
            # Tiene valor específico (ej: "Amueblado: Semi-amueblado")
            partes = texto.split(':', 1)
            clave = partes[0].strip()
            valor = partes[1].strip()
            Datos[clave] = valor
        else:
            # Es una característica simple (ej: "Balcón")
            Datos[texto] = True

    # Enlace 
    Datos['Enlace']=url

    return Datos

In [5]:
Propiedades= []
for n in links:
    Casas = scrapping_por_link(n)
    if Casas:
        Propiedades.append(Casas)

Obteniendo información de: https://www.icasas.cl/propiedad/b4c4-ae6b-194a9bc-400a0f616db0-7723
Obteniendo información de: https://www.icasas.cl/propiedad/6344-a796-7cabb976-36a02a0d33ad-3825
Obteniendo información de: https://www.icasas.cl/propiedad/c5c8-905f-198f811-e8ce324c3c89-78e6
Obteniendo información de: https://www.icasas.cl/propiedad/da21-9ccf-197c21a-9893f8b45585-73cd
Obteniendo información de: https://www.icasas.cl/propiedad/d725-a794-19b8b7b-9e90fbbb8acf-7553
Obteniendo información de: https://www.icasas.cl/propiedad/2429-b0ac-19b8766-b51afd16cf7c-77ab
Obteniendo información de: https://www.icasas.cl/propiedad/b1df-98d0-f1e28d5-c4e0a96b4996-4a4c
Obteniendo información de: https://www.icasas.cl/propiedad/5e59-85e0-704f482-fa9d9f282638-4507
Obteniendo información de: https://www.icasas.cl/propiedad/c669-88ff-95bacc67-9df7974a8302-46fd
Obteniendo información de: https://www.icasas.cl/propiedad/a2cf-af13-199bfc6-7e383bc7d7b7-72b1
Obteniendo información de: https://www.icasas.cl

In [6]:
Propiedades= pd.DataFrame(Propiedades)

In [7]:
Propiedades.columns

Index(['Precio UF', 'N Habitaciones', 'Area m2', 'N Baños', 'Comuna',
       'Provincia', 'Amueblado', 'Gastos Comunes', 'Área útil',
       'Estacionamiento', 'm2 terreno', 'Enlace', 'Año de construcción',
       'Agua', 'Aire Acondicionado', 'Alarma', 'Bodega', 'Chimenea',
       'Placares', 'Cocina Equipada', 'Terraza', 'Portero',
       'Centros comerciales', 'Cerca de Escuelas', 'Cocina Integrada',
       'Electricidad', 'Internet', 'Patio', 'Aseos', 'Piscina', 'Quincho',
       'Cerca de estación de tren', 'Calefacción', 'Jacuzzi', 'Jardín',
       'Estado de conservación', 'Gas Natural', 'Área juegos infantiles',
       'Gimnasio', 'Seguridad 24 horas', 'Vista panorámica', 'Balcón',
       'Pieza de Servicio', 'Acceso a discapacitados', 'Cancha de Tenis',
       'Nivel', 'Sólo familias', 'Sauna', 'Ascensor'],
      dtype='object')

In [8]:
Propiedades.head()

,Precio UF,N Habitaciones,Area m2,N Baños,Comuna,Provincia,Amueblado,Gastos Comunes,Área útil,Estacionamiento,...,Seguridad 24 horas,Vista panorámica,Balcón,Pieza de Servicio,Acceso a discapacitados,Cancha de Tenis,Nivel,Sólo familias,Sauna,Ascensor
0,12.800,3,139,4,Concón,Valparaíso,Sin amueblar,: true,139,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23.000,4,450,5,Viña del Mar,Valparaíso,Sin amueblar,NaN,450,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"8.081,8",4,160,3,Viña del Mar,Valparaíso,Sin amueblar,NaN,160,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.100,3,160,3,San Felipe,San Felipe de Aconcagua,Semi-amueblado,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"9.567,24",4,194,2,Quilpué,Valparaíso,Sin amueblar,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
Propiedades.to_csv(r'C:\Users\sebit\OneDrive\Documentos\Portafolio\Cobre\Casas.csv',sep=',', index=False, encoding='utf-8')